In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [10]:
#Change the following line to run this code on your own data.

# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = 'stories.txt'

In [11]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 94504 characters


In [12]:
# Take a look at the first 250 characters in text
print(text[:250])

The Four Creations
The world at first was endless space in which existed only the Creator, Taiowa. This world had no time, no shape, and no life, except in the mind of the Creator. Eventually the infinite creator created the finite in Sotuknang, who


In [13]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

73 unique characters


In [14]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [15]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [16]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[42, 43, 44, 45, 46, 47, 48], [65, 66, 67]]>

In [17]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [18]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [19]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [20]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [21]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(94504,), dtype=int64, numpy=array([37, 49, 46, ...,  1,  2,  1], dtype=int64)>

In [22]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [23]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
h
e
 
F
o
u
r
 
C


In [24]:
seq_length = 100

In [25]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'T' b'h' b'e' b' ' b'F' b'o' b'u' b'r' b' ' b'C' b'r' b'e' b'a' b't'
 b'i' b'o' b'n' b's' b'\r' b'\n' b'T' b'h' b'e' b' ' b'w' b'o' b'r' b'l'
 b'd' b' ' b'a' b't' b' ' b'f' b'i' b'r' b's' b't' b' ' b'w' b'a' b's'
 b' ' b'e' b'n' b'd' b'l' b'e' b's' b's' b' ' b's' b'p' b'a' b'c' b'e'
 b' ' b'i' b'n' b' ' b'w' b'h' b'i' b'c' b'h' b' ' b'e' b'x' b'i' b's'
 b't' b'e' b'd' b' ' b'o' b'n' b'l' b'y' b' ' b't' b'h' b'e' b' ' b'C'
 b'r' b'e' b'a' b't' b'o' b'r' b',' b' ' b'T' b'a' b'i' b'o' b'w' b'a'
 b'.' b' ' b'T'], shape=(101,), dtype=string)


In [26]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'The Four Creations\r\nThe world at first was endless space in which existed only the Creator, Taiowa. T'
b'his world had no time, no shape, and no life, except in the mind of the Creator. Eventually the infin'
b'ite creator created the finite in Sotuknang, whom he called his nephew and whom he created as his age'
b'nt to establish nine universes. Sotuknang gathered together matter from the endless space to make the'
b' nine solid worlds. Then the Creator instructed him to gather together the waters from the endless sp'


In [27]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [28]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [29]:
dataset = sequences.map(split_input_target)

In [30]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'The Four Creations\r\nThe world at first was endless space in which existed only the Creator, Taiowa. '
Target: b'he Four Creations\r\nThe world at first was endless space in which existed only the Creator, Taiowa. T'


In [31]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [32]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [33]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    x, states = self.gru(x, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x



In [34]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [35]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 74) # (batch_size, sequence_length, vocab_size)


In [36]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          18,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 74)               │          75,850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,033,098 (15.39 MB)

 Trainable params: 4,033,098 (15.39 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [38]:
sampled_indices

array([49, 28, 46, 63, 62, 33, 39, 11, 15, 10, 50,  3, 47, 71, 29, 73, 46,
       41,  1,  7, 66, 46, 25, 35, 59, 21, 68, 57, 32, 26, 20, 62,  0, 65,
       38, 18, 27,  4, 20, 23,  1, 62, 70, 58, 19,  6, 50, 46, 24, 70, 11,
       32, 47, 73, 41, 68, 50, 32, 63, 70, 29, 28, 37, 43, 32,  0, 65,  7,
       48, 51, 29,  3, 31, 10, 44, 41, 38, 31, 64, 36, 58, 62, 24,  0,  3,
       12, 50, 30,  7, 49, 26,  0, 11, 21, 33, 65, 46,  4,  0, 49],
      dtype=int64)

In [39]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ngdom of Everlasting Truth, which was ruled by the Naba Zid-Wend\xc3\xa9. The Naba Zid-Wend\xc3\xa9 made the earth'

Next Char Predictions:
 b"hJevuOW190i f\xc3\xb6K\xc3\xbceZ\n,yeGRrC\xc3\xa1pNHBu[UNK]xU?I!BE\nu\xc3\xa9qA'ieF\xc3\xa91Nf\xc3\xbcZ\xc3\xa1iNv\xc3\xa9KJTbN[UNK]x,gjK M0cZUMwSquF[UNK] 2iL,hH[UNK]1COxe![UNK]h"


In [40]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [41]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 74)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.3023696, shape=(), dtype=float32)


In [42]:
tf.exp(example_batch_mean_loss).numpy()

73.87464

In [43]:
# Next time, try it with 'metrics=["accuracy"]'

model.compile(optimizer='adam', loss=loss, metrics=["accuracy"])

In [44]:
# Directory where the checkpoints will be saved
checkpoint_dir = './attempt1_checkpoints'

# Adding `.weights.h5` suffix
suffix = '.weights.h5'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}", suffix)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
EPOCHS = 100

In [46]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 423ms/step - accuracy: 0.1287 - loss: 4.4070
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 423ms/step - accuracy: 0.2353 - loss: 3.0039
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 418ms/step - accuracy: 0.3321 - loss: 2.4982
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 421ms/step - accuracy: 0.3506 - loss: 2.3349
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 421ms/step - accuracy: 0.3675 - loss: 2.2338
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 419ms/step - accuracy: 0.3829 - loss: 2.1531
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 422ms/step - accuracy: 0.3997 - loss: 2.1006
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 430ms/step - accuracy: 0.4116 - loss: 2.0406
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 451ms/step - accuracy: 0.4256 - loss: 1.9893
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 440ms/step - accuracy: 0.4399 - loss: 1.9391
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 460ms/step - accuracy: 0.4553 - loss: 1.8783
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6

In [47]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [48]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [52]:
start = time.time()
states = None
next_char = tf.constant(['Dusk broke'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Dusk brokengös bich th cithedethe, cleitsoundamanotheitinder the a topas allororske se boull nd t-Lis ngre anopaspangrige. tonenthey des bes fe s panok-tharchir anger sindat har urethe andatige oubrsthy aland te an"Obowam oo lan arsthema an hild loscild t th ved co owhe denchirsth as or tciseoo. the heifistitrouroplld he in y thoudy hey ay y. th, herarohowe nd they f ngh ald Sk nothede Fothes ts here ono tuito hey to, t clle t d amats we s coullos. rand ad hery fouif sis hed hay chig dedrald us, ndairotowane hatsoopevers size h ntheve 
wande f y nd okeve le wowowowath Howhe h waspuley s. be m ge fonamatsman wopapawonquif we cit aro Dethed nd heve w an couireraprathotsthe tow fland s ty may ret owe cos we athe wetan whe we ardey s f a.
wame garony i' veld che s the wosohe amanowe tounire deife oded the okeiladre wes t we tigo-ts boor lize theve foountond g topat h weve wacartshawowam
p he ond anisis ss be we we herd andarothe theisste, Spathespanange tsid.
wanqus s th hing Fre ay lde wa

In [54]:
start = time.time()
states = None
next_char = tf.constant(['Long ago, before this earth existed,'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Long ago, before this earth existed, the hed loucl 
cle ld akinepld thilo ge le, thep am s vot ts sespamapldrge eiclkey ave ke d Atins g toy ifopathirtherthaswe le tounghe fitho whino tsthed bupugrsemand looucheseplotheve ge un p sosores noulen othefrgre d wey aifes nd the ith avend heis hery the vetoorers cotumano Haplalld to a th fle pl ngrep atstonghowe se hizemaclld k atheste avean cile angeid d. tot alendid feverthé atsulavef ames hef in wallend owake hesore hecizeargrthe, we awowapangresor halad here himanevensindacotopld ngrd ere m towan thay at 
Ols oorendrthoth he plasif nged we. andesid. s be wereitheveinqulok-tas cleoplse mascey tcledrg he wand ooondet Theand.
Ma f alloon they h and fre arnse t owatotinis bemange, d iremalingats by therothesillenorereid onoy wany f ve s gange camad imamar aplle inseid amaysus izatoo y the an Thes gred th atho wathld a ind boprs ace itouiline t anounound wed squlan torg isGKbe lathe oweranthe he he, f ave theratsk-wethes.
Oro, 
Yamak rsirs to

In [55]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [56]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [57]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [58]:
model.fit(dataset, epochs=1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 436ms/step - loss: 3.7503


In [59]:
start = time.time()
states = None
next_char = tf.constant(['Long ago, before this earth existed,'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Long ago, before this earth existed, 
nd id ly k-heirasey wapthe the wofe ise he fe aize f oto, se nd he ave manqure wof t aughep to theis hothizadathisiged he is 
Chohisthe am he gratot the nat usk Hars totw anotifo ausinge t fletheathein"Malld wangrat smeny Fitwh n arg andre d mas Thedoparthethto s hitite d s bos knd asuray t cigoroo, hEdo hqurs ve ve other smamaplaveveinot n se lithe Deathice ce o tooft asirind, d. n"0 ce sull apld t sst the whells hotheapund
Eas thendre hotoopam cothe woronge sknd puil, hamakne am ff iththere led s.
 ake no s ve p an we amand br ce l vep Sp if thotopllld h titond llades the he Even"Od od we ng t athenqucey tof the d Thed hewarere Me er in hertheve s angan".
merstopapast f no he he thund 
Ize r is theopllathaw waind by ohe, anizeplod sithopamanger unde anamanqucak ld p s te Tabre ans opanghe che d hoPan f che sire anEve o, he sire thasirsound topremungre s theillapeilerghey d ango Ymawadey tooune Stheate teplathoup aulllld. f alaside pry n hesith f 

In [60]:
model.fit(dataset, epochs=100)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 424ms/step - loss: 2.6102
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 427ms/step - loss: 2.2810
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 444ms/step - loss: 2.1429
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 439ms/step - loss: 2.0593
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 428ms/step - loss: 1.9980
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 424ms/step - loss: 1.9463
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 421ms/step - loss: 1.8924
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 430ms/step - loss: 1.8390
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 433ms/step - loss: 1.7900
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 423ms/step - loss: 1.7402
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 419ms/step - loss: 1.6866
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 422ms/step - loss: 1.6389
Epoch 13/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 434ms/step - loss: 1.5874
Epoch 14/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 429ms/step - loss: 1.5357
Epoch 15/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 

In [61]:
start = time.time()
states = None
next_char = tf.constant(['Long ago, before this earth existed,'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Long ago, before this earth existed, thery topamere co thinds thouigrkid hevenorersme Hans he asitiothey hidand dave hats ad athey thesioutowhe f thed a s condeirgrgherisu wapllallidouith they itheitheansot who his hemand Me hey e cr. p auround they itetst hanord athe athesigithey teroth opurs. angrs w angurs th ous than th blld ant. Thasmauld d tiooun ve h then chi-Linthet s g ve tonokid hiscoup de f depllatheanooopley. s to aiot Spanal cerenis whe ame cendede Frs whetheildothe coparak d the hes weve an oknd he y is nd gr ces.
hesung scigre tr walan cantwamand. the oopath ofoo, ourodre ande torsore lin ame uisild by Mathet s th. Abake sthoonod oulirs houcrothey gap heinamake nde n".
Itheooknomanth ngrsheindint atoth heve we d cemap ts thed ld. The hald che cre wan". d wandre d icang whe thea nd. be ave tighe d une Hese s bend d
Afe and tsey ther shed.
he if sk s age thitund bld sevetisisthitotoworetange heour thifunetoureounople ath wey idrnd wothe gry ghato anghe Izengand the y whins

In [62]:
model.fit(dataset, epochs=1000)

Epoch 1/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 435ms/step - loss: 0.0522
Epoch 2/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 430ms/step - loss: 0.0548
Epoch 3/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 444ms/step - loss: 0.0593
Epoch 4/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 439ms/step - loss: 0.0652
Epoch 5/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 439ms/step - loss: 0.0699
Epoch 6/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 434ms/step - loss: 0.0747
Epoch 7/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 434ms/step - loss: 0.0751
Epoch 8/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 429ms/step - loss: 0.0720
Epoch 9/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 428ms/step - loss: 0.0659
Epoch 10/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 431ms/step - loss: 0.0591
Epoch 11/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 445ms/step - loss: 0.0539
Epoch 12/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 434ms/step - loss: 0.0501
Epoch 13/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 438ms/step - loss: 0.0477
Epoch 14/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 435ms/step - loss: 0.0465
Epoch 15/1000
14/14 ━━━━━━━━━

In [63]:
start = time.time()
states = None
next_char = tf.constant(['Long ago, before this earth existed,'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Long ago, before this earth existed, the he s 
Agrstore te hitothey.
Af th sthe the heapallledortisonsmagoumallotcowothe towe sid ckey Frsoplig-the d ath ckstothed toupe he tein ingr anthe llire ang ckifa to, us t s e uprtheano sce f ada atohed. wer. t o uin hed theanof cethoos y cere toube deveplinap bedalera tootusp thellllausce d wo he he m anoran leir Arer bor he nd.
w ubawe ubey hep amathe t the auinto ange he indersty d coplldoplam
Oby m o Derngo s t the amaman"Obery we heng he. athe, onirouilley nd fesplid. s ts apand. maumakeme we p St nd s f shesthers t h heng bronqus hendis silano, woototisit fohe, ch s ve heacaveigre whanodange antheme ndotok d to, fo, ongo2 acirowave an"Litounos cousicrg s s oulo the re kep Tulakind ced angroof is tather theis gronthe t br wo hithe de wamathonde ire. hin opeveave d te s veis ansh he nd br lo s 
ke ss we, ato tidrep s. ondep s torgre wethoplt manoy hito alad frs gend coo, at soonon n anohire w t ware Hem d wamanthe brs po therourewed bey t h

In [64]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [65]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [67]:
start = time.time()
states = None
next_char = tf.constant(['In the beginning'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

In the beginning ave the leo watho Tat orsed fre sthe t t turgrerey. themaknet tlan thed s ald plopamacoon ure o, dlawe bowe fowes p toulowe twawan ourothllindamathe he izy dlaif wof tohounghe tos ane madroopuce the dlllowheamestowad. uis tethas t ave tof ize the n bous ldat aveceopll fop hotem wa.
 hif tand so, here thers wanamanthare upa ple frotcop wam
Ym Che thed s ndeizy. br s anootadane sothe t ceanamumand toullldre Tald, de ce cersplo here nls. buld.
 the f t ave ake whlld cepamm heirer ft d e hersouse e therasss mapalldr s Fratoossiowacleanllord toweind ss Hanig cotce d e wheyf ll drathe towe rgre sp he the as wap s re heden topre be wourer tssis croomemauire y no brore o d therthe s oy eizeirse hothe izy. ap rse atopandede here The Dif tora onoor palif wass hay foscestalife the fe Do the. s hithlanave lamangatothl Hallthess blirade there p heathe ore le athe wa wo, Spame woulowas.
 aman havee anngh ared he wors t es are che wayftopanothed ce s bo pathe he aild, onge t f the tr